## Getting data from Spotify API to get features for prediction

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import spotipy
import spotipy.util as util
import json

# from functions import *
import pickle
import json
from datetime import datetime

In [ ]:
# reading in credentials necessary to use the API
# remember to save credentials in double quotes or else it gets mad at you
with open('../credentials.json') as filename:
    credentials = json.load(filename)
    filename.close()

In [ ]:
token = util.oauth2.SpotifyClientCredentials(client_id=credentials['client_id'],
                                             client_secret=credentials['client_secret'])
## creates an access token for you to do what you need to do
cache_token = token.get_access_token()
# probably need to check how much access it allows on default and if you need more access to do more interesting things


In [ ]:
# need to create an object
sp = spotipy.Spotify(cache_token)

### Loading in data and search queries to use the API

In [ ]:
data = pd.read_csv('data/billboards.csv')
data.rename(columns={
    'Artist': 'artist',
    'Song':'song'
}, inplace=True)
data['search_queries'] = (data.artist +' '+ data.song).str.lower()

In [ ]:
def get_features(df):
    results_list = []
    for queries in df.search_queries.values:
        test = sp.search(q=queries, type = 'track', limit=1)
        results = test['tracks']['items']
        cleaned = clean_search_results(results) # returns a dictionary
        cleaned['search_query'] = queries
        results_list.append(cleaned)
    yield list(results_list)

In [ ]:
with open('output.pkl', 'wb') as filename:
    results_list=[]
    results_list.extend([x for x in get_features(data)]) # testing on the head only
    pickle.dump(results_list[0], filename)
    filename.close()

In [ ]:
len(results_list[0])

In [ ]:
test = pickle.load(open('../data/output.pkl', 'rb'))
test_df = pd.DataFrame(test)
pickle.dump(test_df, open('../data/spotify_api_search_results.pkl', 'wb'))

In [ ]:
output_data = pickle.load(open('../data/spotify_api_search_results.pkl', 'rb')) # NTS: i need to look back and review what popularity is

In [ ]:
from collections import Counter

len(Counter(output_data['id'].values)), len(output_data['id'].values)

In [ ]:
def get_audio_features(id_value):
    try:
        results = sp.audio_features(id_value)[0]
        results['id'] = id_value
    except Exception as e:
        results = {}
        results['id'] = 'missing'
    return results

In [ ]:
audio_features = output_data['id'].apply(get_audio_features)
pickle.dump(audio_features, open('data/audio_features.pkl', 'wb'))

In [ ]:
sp.track('6i0V12jOa3mr6uu4WYhUBr').keys()

NTS: it seems like the track method returns results that I already have stores so i'm not going to bother with that right now.... might be something to look into later when i come back to the project?

In [ ]:
sp.audio_analysis('6i0V12jOa3mr6uu4WYhUBr')['track'].keys()

In [ ]:
def get_audio_analysis(song_id):
    """
    takes the song id and passes it through the spotify api to get the song analysis values
    and if there is no song_id then it passes and returns an empty string
    """
    relevant_columns = ['num_samples', 'duration', 'sample_md5', 'offset_seconds',
                             'window_seconds', 'analysis_sample_rate', 'analysis_channels',
                             'end_of_fade_in', 'start_of_fade_out', 'loudness', 'tempo',
                             'tempo_confidence', 'time_signature', 'time_signature_confidence',
                             'key', 'key_confidence', 'mode', 'mode_confidence']
    try:
        song_analysis = sp.audio_analysis(song_id)['track']
        result = {key:song_analysis[key] for key in song_analysis if key in relevant_columns}
    except Exception as e:
        print(e)
        result = {key:'' for key in relevant_columns}
    result['id'] = song_id
    return result
        

In [ ]:
## i think i also remember being able to get features on the artist tooo. i should explore whatt other ttings are avaiable in the api

In [ ]:
analysis_df = pd.DataFrame.from_dict(output_data['id'].apply(get_audio_analysis))
analysis_df.head()

In [ ]:
from datetime import datetime

query_date = str(datetime.today().month).zfill(2) + '_' + str(datetime.today().day).zfill(2)
print(query_date)
pickle.dump(analysis_df, open(f'analysis_query_{query_date}.pkl', 'wb'))

### NTS: I want to get the release date of the songs too 